In [8]:
import pandas as pd
import requests
import json
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime
import pandas as pd
import requests
import os
from datetime import datetime, timedelta
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders

headers = {
  "authority": "www.irishjobs.ie",
  "method": "GET",
  "path": "/membersarea/api/recommendedjobs?count=20",
  "scheme": "https",
  "accept": "*/*",
  "accept-encoding": "gzip, deflate, br, zstd",
  "accept-language": "en-US,en;q=0.9,hi;q=0.8",
  "cookie": "VISITOR_ID=25a8fb16663ce6023a77af83f5245f09; anonymousUserId=e46cc09b-2459-147a-2c91-21bd01f709bb; sc_vid=6b3c82139b85af99483d257b54b30a63; CONSENTMGR=c1:0%7Cc2:0%7Cc3:0%7Cc4:0%7Cc5:0%7Cc6:0%7Cc7:0%7Cc8:0%7Cc9:1%7Cc10:0%7Cc11:0%7Cc12:0%7Cc13:0%7Cc14:0%7Cc15:0%7Cts:1759618350016%7Cconsent:true; STEPSTONEV5LANG=en; SoftLoginCookie=DL92limBnSx%2BeWan2Q9j6c03LZ6fLxl%2BUQtSSkcX%2Bm%2BtF4ojTTNsUgGc01RYxdey1XWCDes2aBkCk0Wr5lL6Yk5xDrxmPfNVBXSxK1PKYGNdj3d0gKcNdG8mICzMYaKapvz7zM5UU7Zwvcn9Ax5slNhcEGuEQEoI9%2FuIopUGX9pjYS%2F6NhA7%2BP28xg4IlI2382CSteFs22I4Bb36yjhYsF0wD542rc4YB15UV3LzOpU%3D; AnonymousUser=MemberId%3D4b60357d-35fd-4560-b8ef-87c2df731142%26IsAnonymous%3DFalse; ... (truncated for readability)",
  "priority": "u=1, i",
  "sec-ch-ua": "\"Google Chrome\";v=\"141\", \"Not?A_Brand\";v=\"8\", \"Chromium\";v=\"141\"",
  "sec-ch-ua-mobile": "?1",
  "sec-ch-ua-platform": "\"Android\"",
  "sec-fetch-dest": "empty",
  "sec-fetch-mode": "cors",
  "sec-fetch-site": "same-origin",
  "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Mobile Safari/537.36"
}


# -----------------------------
# 1️⃣ Scrape IrishJobs API
# -----------------------------

URL = 'https://www.irishjobs.ie/membersarea/api/recommendedjobs?count=500'

resp = requests.get(URL, headers=headers)
j = resp.json()

data = [i for i in j['recommendedJobs']]
df = pd.DataFrame(data)

# -----------------------------
# 2️⃣ Keep only active jobs
# -----------------------------
df = df[df["isActive"] == True]

# -----------------------------
# 3️⃣ Classify job type
# -----------------------------
def classify_job_type(row):
    t = str(row["title"]).lower()
    s = str(row["salary"]).lower()

    if "part time" in t or "part-time" in t:
        return "Part-Time"
    if "full time" in t or "full-time" in t or "permanent" in t:
        return "Full-Time"
    if "temporary" in t or "temp" in t or "contract" in t or "fixed term" in t:
        return "Temporary/Contract"
    if "seasonal" in t or "christmas" in t or "festive" in t:
        return "Seasonal"
    if "per annum" in s or "annually" in s or "yearly" in s:
        return "Full-Time"
    if "per hour" in s or "hourly" in s:
        return "Part-Time"
    return "Other"

df["job_type"] = df.apply(classify_job_type, axis=1)

# -----------------------------
# 4️⃣ Standardize county
# -----------------------------
# Mapping of towns/suburbs to counties
location_map = {
    # Counties
    "county dublin": "Dublin",
    "county meath": "Meath",
    "county cork": "Cork",
    "county kildare": "Kildare",
    "county armagh": "Armagh",
    "county wicklow": "Wicklow",
    "county louth": "Louth",

    # Full location formats
    "dublin, county dublin": "Dublin",
    "cork, county cork": "Cork",
    "kildare, county kildare": "Kildare",
    "armagh, county armagh": "Armagh",
    "bray, county wicklow": "Bray, Wicklow",
    "craigavon, county armagh": "Craigavon, Armagh",

    # Towns/suburbs (Place + County)
    "south dublin, dublin": "South Dublin, Dublin",
    "lucan and pettycanon": "Lucan And Pettycanon, Dublin",
    "palmerstown": "Palmerstown, Dublin",
    "clane": "Clane, Kildare",
    "newbridge": "Newbridge, Kildare",
    "swords": "Swords, Dublin",
    "blanchardstown": "Blanchardstown, Dublin",
    "celbridge, kildare": "Celbridge, Kildare",
    "malahide": "Malahide, Dublin",
    "leixlip": "Leixlip, Kildare",
    "rathcoole": "Rathcoole, Dublin",
    "tallaght": "Tallaght, Dublin",
    "summerhill": "Summerhill, Meath",
    "collegeland": "Collegeland, Armagh",
    "clondalkin": "Clondalkin, Dublin",
    "cornelscourt": "Cornelscourt, Dublin",
    "maynooth, kildare": "Maynooth, Kildare",
    "dublin city centre": "Dublin City Centre, Dublin",
    "greenoge": "Greenoge, Offaly",
    "fermoy, cork": "Fermoy, Cork",
    "balbriggan": "Balbriggan, Dublin",
    "mitchelstown, cork": "Mitchelstown, Cork",
    "ringaskiddy": "Ringaskiddy, Cork",
    "little island": "Little Island, Cork",
    "macroom, cork": "Macroom, Cork",
    "stillorgan north": "Stillorgan North, Dublin",
    "kimmage": "Kimmage, Dublin",
    "dun laoghaire": "Dun Laoghaire, Dublin",
    "dunmanway south": "Dunmanway South, Cork",
    "naas, kildare": "Naas, Kildare",
    "santry": "Santry, Dublin",
    "portmarnock": "Portmarnock, Dublin",
    "saggart": "Saggart, Dublin",
    "mallow, cork": "Mallow, Cork",
    "bishopstown": "Bishopstown, Cork",
    "carrigaline": "Carrigaline, Cork",
    "republic of ireland": "Other"
}

def standardize_county(location):
    if not isinstance(location, str):
        return "Other"

    loc = location.lower().strip()

    if loc in location_map:
        return location_map[loc]

    # Handle "Area, County X" general case
    for county in ["cork", "dublin", "kildare", "meath", "armagh", "wicklow", "louth", "limerick", "kerry", "offaly"]:
        if county in loc:
            place = loc.split(',')[0].strip().title()
            if "county" in place.lower():
                return county.capitalize()
            else:
                return f"{place}, {county.capitalize()}"

    return "Other"

# Example usage with a list
locations = [
    'County Dublin', 'Dublin, County Dublin', 'County Meath',
    'South Dublin, Dublin', 'Lucan And Pettycanon',
    'Cork, County Cork', 'Palmerstown', 'Clane',
    'Kildare, County Kildare', 'County Cork', 'Newbridge', 'Swords',
    'Blanchardstown', 'Armagh, County Armagh', 'Celbridge, Kildare',
    'Craigavon, County Armagh', 'Malahide', 'Leixlip', 'Rathcoole',
    'Tallaght', 'Summerhill', 'Collegeland', 'Clondalkin',
    'Cornelscourt', 'Maynooth, Kildare', 'Bray, County Wicklow',
    'Dublin City Centre', 'Greenoge', 'Fermoy, Cork', 'County Louth',
    'Balbriggan', 'Mitchelstown, Cork', 'Ringaskiddy', 'Little Island',
    'Macroom, Cork', 'Stillorgan North', 'Kimmage', 'Dun Laoghaire',
    'Dunmanway South', 'Naas, Kildare', 'Santry', 'Portmarnock',
    'Saggart', 'Republic of Ireland', 'Mallow, Cork', 'Bishopstown',
    'Carrigaline'
]

standardized_locations = [standardize_county(loc) for loc in locations]

for orig, std in zip(locations, standardized_locations):
    print(f"{orig} -> {std}")

# Apply to DataFrame
df["county"] = df["location"].apply(standardize_county)

# -----------------------------
# 5️⃣ Simplify location and drop unnecessary columns
# -----------------------------
df['location'] = df["county"]
df.drop(columns=["external","companyLogoAlt","companyLogo","companyId",
                 "companyUrl","listingGlobalId","id","applyUrl"], inplace=True)

# -----------------------------
# 6️⃣ Save all jobs with timestamp
# -----------------------------
save_folder = r"C:\Users\Chandra\OneDrive\Documents\Job_file_alert"  # change as needed
os.makedirs(save_folder, exist_ok=True)
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
all_file = os.path.join(save_folder, f"irish_jobs_all_{timestamp}.xlsx")
df.to_excel(all_file, index=False)
print(f"All jobs file saved: {all_file}")

# -----------------------------
# 7️⃣ Filter jobs posted in last 2 hours
# -----------------------------
df['datePosted'] = pd.to_datetime(df['datePosted']).dt.tz_localize(None)
now = datetime.now()
recent_jobs = df[df['datePosted'] >= (now - timedelta(hours=2))]

columns_to_keep = ["title", "companyName", "location", "salary",
                   "datePosted", "dateExpire", "url", "isActive", "job_type"]
recent_jobs = recent_jobs[columns_to_keep]

# Save recent jobs
recent_file = os.path.join(save_folder, f"irish_jobs_recent_{timestamp}.xlsx")
recent_jobs.to_excel(recent_file, index=False)
print(f"Recent jobs file saved: {recent_file}")

# -----------------------------
# 8️⃣ Send email with recent jobs Excel
# -----------------------------
def send_email_with_excel(file_path):
    if recent_jobs.empty:
        print("No recent jobs in last 2 hours. Email not sent.")
        return

    sender_email = "Chandra.binod.chetry@gmail.com"
    receiver_email = "Chandra.binod.chetry@gmail.com"
    password = "nlbc kicb qmmb icln"

    subject = "New Irish Jobs in the last 2 hours"
    body = "Please find attached the latest jobs posted in the last 2 hours."

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Attach Excel file
    with open(file_path, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header(
            "Content-Disposition",
            f"attachment; filename= {os.path.basename(file_path)}",
        )
        msg.attach(part)

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender_email, password)
    server.send_message(msg)
    server.quit()
    print(f"Email sent with {file_path} attached!")

send_email_with_excel(recent_file)




County Dublin -> Dublin
Dublin, County Dublin -> Dublin
County Meath -> Meath
South Dublin, Dublin -> South Dublin, Dublin
Lucan And Pettycanon -> Lucan And Pettycanon, Dublin
Cork, County Cork -> Cork
Palmerstown -> Palmerstown, Dublin
Clane -> Clane, Kildare
Kildare, County Kildare -> Kildare
County Cork -> Cork
Newbridge -> Newbridge, Kildare
Swords -> Swords, Dublin
Blanchardstown -> Blanchardstown, Dublin
Armagh, County Armagh -> Armagh
Celbridge, Kildare -> Celbridge, Kildare
Craigavon, County Armagh -> Craigavon, Armagh
Malahide -> Malahide, Dublin
Leixlip -> Leixlip, Kildare
Rathcoole -> Rathcoole, Dublin
Tallaght -> Tallaght, Dublin
Summerhill -> Summerhill, Meath
Collegeland -> Collegeland, Armagh
Clondalkin -> Clondalkin, Dublin
Cornelscourt -> Cornelscourt, Dublin
Maynooth, Kildare -> Maynooth, Kildare
Bray, County Wicklow -> Bray, Wicklow
Dublin City Centre -> Dublin City Centre, Dublin
Greenoge -> Greenoge, Offaly
Fermoy, Cork -> Fermoy, Cork
County Louth -> Louth
Balbrig

In [4]:
df["location"]=='other'

,location
0,False
1,False
2,False
3,False
4,False
...,...
495,False
496,False
497,False
498,False
